# Apache Druid and Log4j

Apache Druid uses Log4J to emit information as it runs. They not only enable you to investigate issues and solve problems, but to understand how each of Druid's processes work in isolation and in collaboration with one another.

In this learning module we will:

* Identify the various Druid process log files.
* Understand the role of the log files.
* Review some task log files.

The first step in making use of log files is to become aware of what logs are available. We'll see that the Druid processes each generate a couple of different files. In addition to the process log files, we will see that transient Druid worker tasks also generate log files.

Apache Druid has a microservices architecture, enabling scale and resilience. Logs are an essential source of information when troubleshooting and monitoring a cluster.

As Druid processes run, they write status information into files called log files. We can use these files to understand the Druid processes' behaviors and diagnose problems.



Some processes may spin off tasks to perform sub-processing. In Druid, a task is separate process that usually runs in its own JVM. Each of these tasks create their own log files.

Many of the logs capture behavior during ingestion and other processing, but we can also configure Druid to capture specific query information.

## Prerequisites

This tutorial works with Druid 28.0.1 or later.

### Run with Docker

Launch this tutorial and all prerequisites using the `jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see the Learn Druid repository [readme](https://github.com/implydata/learn-druid).

## Initialization

To use this notebook, you will need to run Druid locally.

You will also make extensive use of the terminal, which we suggest you place alongside this notebook or on another screen.

### Install tools

Open a local terminal window.

If you have not already install `wget` or `multitail`, run the following commands to install these tools using `brew`.

```bash
brew install multitail && brew install wget
```

Run the following command to pull the default configuration for `multitail` to your home folder. Do not run this command if you are already running `multitail` as it will overwrite your own configuration.

```bash
curl https://raw.githubusercontent.com/halturin/multitail/master/multitail.conf > ~/.multitailrc
```

### Install Apache Druid

Run the following to create a dedicated folder for learn-druid in your home directory:

```bash
cd && mkdir learn-druid-local
cd learn-druid-local
```

Now pull a compatible version of Druid.

> If you do not have wget on your Mac, you can install it with brew.

```bash
wget https://dlcdn.apache.org/druid/28.0.1/apache-druid-28.0.1-bin.tar.gz && tar -xzf apache-druid-28.0.1-bin.tar.gz &&
cd apache-druid-28.0.1
```

# Review the log file configuration

The log file configuration is set in the `log4j2.xml` alongside Druid's own configuration files.

Remembering that Druid has multiple configuration file locations out-of-the-box, run this command to view the `auto` configuration file for logs that's used by the `learn-druid` script:

```bash
more ~/learn-druid-local/apache-druid-28.0.1/conf/druid/auto/_common/log4j2.xml
```

On the first page, within the [`Configuration`](https://logging.apache.org/log4j/log4j-2.4/manual/configuration.html#ConfigurationSyntax) you will see:

* [`Properties`](https://logging.apache.org/log4j/2.x/manual/configuration.html#PropertySubstitution) provide key/values pairs that may be used throughout the configuration file.
* [`Appenders`](https://logging.apache.org/log4j/2.x/manual/appenders.html) designate the format of log messages and determine the target for the messages.

Further along you will see:

* [`Loggers`](https://logging.apache.org/log4j/2.x/manual/configuration.html#Loggers) filter the log messages and dispense them to Appenders. Loggers can filter messages based on the Java package and/or class and by message priority.

The default Configuration for Druid does not include a [`monitorInterval`](https://logging.apache.org/log4j/log4j-2.4/manual/configuration.html#AutomaticReconfiguration) property, so changes to logging configuration are only recognised when a process restarts.

For the purposes of this tutorial, run the following command to add a `monitorInterval` property to the Configuration.

```bash
sed -i '' 's/<Configuration status="WARN">/<Configuration status="WARN" monitorInterval="5">/' \
  ~/learn-druid-local/apache-druid-28.0.1/conf/druid/auto/_common/log4j2.xml
```
Now start Druid with the following command, and remember to take a note of the process Id reported after the command has run:

```bash
nohup bin/start-druid > log.out 2> log.err < /dev/null & disown
```

__It's important that you remember this process ID for later in the tutorial.__

## Properties

In Druid, `Properties` are leveraged to set a location for all logs. This location is calculated when Druid stars, and passed as a parameter to Java and on to Log4J for each process.

> IS THE ABOVE CORRECT?

By default, this location is a "log" folder at the root of your Druid installation, but [can be over-ridden]((https://druid.apache.org/docs/latest/configuration/logging#log-directory) by using the `DRUID_LOG_DIR` system variable.

## Appenders

There are two `appenders`:

* A [`Console`](https://logging.apache.org/log4j/log4j-2.4/manual/appenders.html#ConsoleAppender) appender called "Console" for `SYSTEM_OUT`.
* A [`RollingRandomAccessFile`](https://logging.apache.org/log4j/log4j-2.4/manual/appenders.html#RollingRandomAccessFileAppender) appender called "FileAppender" which is used for detailed process logs.

The "FileAppender" `RollingRandomAccessFile` appender has `fileName` and `filePattern` properties. These control the initial and on-going file names.

Run the following command to see switch to the default location for the log files.

```bash
cd ~/learn-druid-local/apache-druid-28.0.1/log
```

Since Druid is a distributed system, we will find log files for each Druid process. In addition, Druid also captures the output written to the standard output.

Use the following command to take a look at what is here:
```bash
ls
```

You can see how this results in two sets of files being created:

* `process name.stdout.log` file - which is the information written by the processes to stdout (i.e., the terminal)
* `process name.log` - file containing various status, error, warning and debug messages

### Log filenames

The `fileName` property of the "FileAppender" sets the name of the log file being written to at the moment, while `filePattern` is applied when the log rolls over.

Run the following command to change the default filename in the log4j configuration file:

```bash
sed -i '' 's/{sys:druid.node.type}.log/{sys:druid.node.type}-mycluster.log/' \
  ~/learn-druid-local/apache-druid-28.0.1/conf/druid/auto/_common/log4j2.xml
```

List the contents of the log folder again to see the changes:

```bash
ls
```

Revert the change with the following command:

```bash
sed -i '' 's/{sys:druid.node.type}-mycluster.log/{sys:druid.node.type}.log/' \
  ~/learn-druid-local/apache-druid-28.0.1/conf/druid/auto/_common/log4j2.xml
```

Since "FileAppender" is a [RollingRandomAccessFileAppender](https://logging.apache.org/log4j/log4j-2.4/manual/appenders.html#RollingRandomAccessFileAppender) you can adjust the `TimeBasedTriggeringPolicy` to change when `fileName` log files are rolled over to `filePattern` log files.

Run the following in your terminal to adjust the `filePattern` to include the hours and minutes.

```bash
sed -i '' 's/{yyyyMMdd}/{yyyyMMdd-HH:mm}/' \
  ~/learn-druid-local/apache-druid-28.0.1/conf/druid/auto/_common/log4j2.xml
```

Since the `TimeBasedTriggeringPolicy` is set to 1 by default, a change in the least granular element of the `filePattern` will trigger a rollover. In this case, you will now see a new file being created every minute.

```bash
ls
```

### Log retention

Since "FileAppender" is a [RollingRandomAccessFileAppender](https://logging.apache.org/log4j/log4j-2.4/manual/appenders.html#RollingRandomAccessFileAppender) you can adjust the `DefaultRolloverStrategy` to control retention of logs by adjusting the `Delete` section.

`IfFileName` and `IfLastModified` are used in conjunction to remove any files from the log folder that match the rules. The default is to remove matching files older than two months, based on the date last modified.

### Log patterns

The content of log files is specified in the `PatternLayout`.

```
`%d{ISO8601} %p [%t] %c -%notEmpty{ [%markerSimpleName]} %m%n`
```

* Timestamp (`%d{ISO8601}`)
* Message priority (`%p`)
* Thread name (`[%t]`)
* Class name (`%c`)
* Message (`%m%n`)

Run this command to see this information from two of the process logs:

```bash
multitail --config multitail.conf -CS log4jnew -du -P a \
    -f coordinator-overlord.log
```

Threads and classes are helpful for diagnosis, especially for WARN and ERROR conditions, while messages describe what has happened, what the state of the process or some significant variable is. These serve not just for diagnosis but for general learning of how the Druid database works.

## Loggers

This section of the configuration controls what types of events are logged and what data as recorded.

### Logging level

Developers assign different log levels to different entries, indicating how severe an event is.

* FATAL (system failure)
* ERROR (functional failure)
* WARN (non-fatal issue)
* INFO (notable event)
* DEBUG (program debugging messages)
* TRACE (highly granular execution event)

The default configuration sets a `Root` level for the `FileAppender` of `INFO`, meaning that only messages with a level of `INFO` and above will be recorded.

```xml
    <Root level="info">
        <AppenderRef ref="FileAppender"/>
    </Root>
```

Other base levels are set at a class level, reducing log noise. For example:

```xml
    <!-- Quieter KafkaSupervisors -->
    <Logger name="org.apache.kafka.clients.consumer.internals" level="warn" additivity="false">
        <Appender-ref ref="FileAppender"/>
    </Logger>
```

Run this command to amend the base logging level for all Druid processes:

```bash
sed -i '' 's/Root level="info"/Root level="debug"/' \
  ~/learn-druid-local/apache-druid-28.0.1/conf/druid/auto/_common/log4j2.xml
```

Watch the change in information on the multitail window.

Revert the logging level to INFO before proceeding.

```bash
sed -i '' 's/Root level="debug"/Root level="info"/' \
  ~/learn-druid-local/apache-druid-28.0.1/conf/druid/auto/_common/log4j2.xml
```

## Using the logs

Two recommended approaches for working with log files are:

1. Search individual log files for indications of a problem, such as WARN, ERROR, and FATAL, and of Java exceptions in stack traces (e.g. `java.net.ConnectionException: Connection refused`.
2. Read log files more like a novel, starting at the beginning of the file - or some known intermediate point - and then follow the story logically.

In approach 1, it's possible to work back through the history, using the time field as a key. You may notice large time gaps, or use filtering to remove any events from threads or classes that you know are not relevant to the error.

Approach 2 requires more time, but helps to solve more complex problems. It's also an important learning aid when delving deep into how Druid's processes collaborate to realise services.

### Useful searches in logs

| Log | Search Term |
|---|---|
| Any | __NodeRoleWatcher__<br>Across all the processes, watch as they detect changes in the processes that are running in the cluster, and see what they do about it! |
| Coordinator / Overlord | __org.apache.druid.metadata.SQLMetadataRuleManager__<br>This is the coordinator polling the rules in the metadata database, getting ready to apply them. The log tells you how many rules it picks up and how many data sources they cover. |
| Coordinator / Overlord | __org.apache.druid.metadata.SqlSegmentsMetadataManager__<br>Messages show how many segments the cluster thinks are “used” – ready to be used for queries. |
| Coordinator / Overlord | __org.apache.druid.indexing.overlord.RemoteTaskRunner__<br>GIves interesting information about what’s happening with ingestion resources in the cluster, including when they first advertise themselves. |
| Coordinator / Overlord | __org.apache.druid.server.coordinator.rules__<br>Lots of information about how the retention rules are actually being applied (or not!). |
| Historical | __org.apache.druid.server.coordination.BatchDataSegmentAnnouncer__<br>You can see individual segments being announced as available for query by each historical server as it loads them. |
| Historical | __org.apache.druid.server.coordination.SegmentLoadDropHandler__<br>As well as seeing how the historical checks its local segment cache on startup, you can watch along as the Historical picks up the instructions from the coordinator and then does something about them. When there are ERRORs like “Failed to load segment for dataSource” you get traces about what the process was trying to do – quite often something pointing to an error with its connection to deep storage. |
| Any | __org.apache.druid.initialization.Initialization__<br>These messages are all about the process starting up. It can be interesting to see what exactly each one does – and if it runs into issues. |
| Coordinator | __org.apache.druid.server.coordinator.duty.BalanceSegments__<br>Here you can see what Druid decides to do when balancing needs to be carried out – e.g. a server is lost or added. |

### Example: failure in Apache Zookeeper

Druid's process rely on [Apache Zookeeper](https://zookeeper.apache.org/) for [inter-process communication and configuration](https://druid.apache.org/docs/latest/dependencies/zookeeper.html). Run the following command to simulate a failure in Zookeeper.

The first of these two commands prevents Zookeeper from being recovered by the `learn-druid` script's `supervisor`, and the second then kills the Zookerper process.

```bash
kill -STOP $(ps -ef | grep 'perl' | awk 'NF{print $2}' | head -n 1)
kill $(ps -ef | grep 'zoo' | awk 'NF{print $2}' | head -n 1)
```

Lets see how this appears in the logs:

```bash
multitail -CS log4jnew -du -P a -s 2 -sn 2,2 \
    -f coordinator-overlord.log \
    -f broker.log \
    -f middlemanager.log \
    -f historical.log
```

Run the command below to have Zookeeper restart.

```bash
kill -CONT $(ps -ef | grep 'perl' | awk 'NF{print $2}' | head -n 1)
```

# Stop Druid

Run this command to stop Druid, replacing "{pid}" with the process Id you noted earlier.

```bash
kill {pid}
```

For example:

```bash
kill 9864
```

> If you do not remember your PID, use `ps` to look for the `supervise` process.

## Learn more

In the lab you learned that you can turn on logging for query requests with the druid.startup.logging.logProperties setting. Read all the options - including other possible targets for these logs - in the documentation. An interesting configuration, for example, automatically filters query logging for you.

Watch this Druid Summit presentation by Amir Youssefi and Pawas Ranjan from Conviva that describes how useful this information can be to tuning Druid clusters.

* [Druid optimizations for scaling customer facing analytics at Conviva](https://youtu.be/zkHXr-3GFJw?t=746)

Take a few minutes to scan the official documentation for information about logging configuration. You may want to keep this page to hand throughout the course.

* [Logging](https://druid.apache.org/docs/latest/configuration/logging.html)

You're about to learn more about Apache Druid's use of Apache Logging Services in the form of Log4J™. Get insight into the background and benefits of Log4J on the official project website:

* [Apache Logging Services](https://logging.apache.org/)
* Read your options for `filePattern`s by refering to the [simple date format](https://docs.oracle.com/javase/7/docs/api/java/text/SimpleDateFormat.html) documentation.